# tut1.py
### tut1.m implemented in Python for cantera (now as a jupyter notebook)
cf. http://www.cantera.org/docs/sphinx/html/matlab/tutorials/tut1.html

Copyleft 2016, i.e. MIT License; Ernest Yeung <ernestyalumni@gmail.com>                            
20160629

# Tutorial 1:  Getting started
   
Topics:
    - creating a gas mixture
    - setting the state
    - cleaning up


In [1]:
import cantera as ct
import numpy as np

In [2]:
gas1 = ct.Solution("gri30.xml")

In [3]:
print gas1()


  gri30:

       temperature             300  K
          pressure          101325  Pa
           density       0.0818891  kg/m^3
  mean mol. weight         2.01588  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy           26470        5.336e+04     J
   internal energy     -1.2109e+06       -2.441e+06     J
           entropy           64914        1.309e+05     J/K
    Gibbs function     -1.9448e+07        -3.92e+07     J
 heat capacity c_p           14312        2.885e+04     J/K
 heat capacity c_v           10187        2.054e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2              1                1         -15.7173
     [  +52 minor]              0                0

None


What you have just done is to create an object ("gas1") that
implements GRI-Mech 3.0, the 53-species, 325-reaction natural gas
combustion mechanism developed by Gregory P. Smith, David M. Golden,
Michael Frenklach, Nigel W. Moriarty, Boris Eiteneer, Mikhail
Goldenberg, C. Thomas Bowman, Ronald K. Hanson, Soonho Song, William
C. Gardiner, Jr., Vitali V. Lissianski, and Zhiwei Qin. (See
http://www.me.berkeley.edu/gri_mech/ for more information about
GRI-Mech 3.0.)

The object created by GI30 has properties you would expect for a gas
mixture - it has a temperature, a pressure, species mole and mass
fractions, etc. As we'll soon see, it has many other properties too.
The summary of the state of 'gas1' printed above shows that new
objects created by function GRI30 start out with a temperature of
300 K, a pressure of 1 atm, and have a composition that consists of
only one species, in this case hydrogen. There is nothing special
about H2 - it just happens to be the first species listed in the
input file defining GRI-Mech 3.0 that the 'GRI30' function reads. In
general, the species listed first will initially have a mole
fraction of 1.0, and all of the others will be zero.

##  Setting the state
  -----------------

The state of the object can be easily changed. For example,

In [4]:
gas1.TD = (1200,None)

In [5]:
gas1()


  gri30:

       temperature            1200  K
          pressure          405300  Pa
           density       0.0818891  kg/m^3
  mean mol. weight         2.01588  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      1.3296e+07         2.68e+07     J
   internal energy      8.3462e+06        1.682e+07     J
           entropy           79510        1.603e+05     J/K
    Gibbs function     -8.2116e+07       -1.655e+08     J
 heat capacity c_p           15378          3.1e+04     J/K
 heat capacity c_v           11253        2.269e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2              1                1         -16.5912
     [  +52 minor]              0                0



sets the temperature to 1200 K. (Cantera always uses SI units.)

Notice in the summary of properties that MATLAB prints after this
command is executed that the temperature has been changed as
requested, but the pressure has changed too. The density and
composition have not. 

When setting properties individually, some convention needs to be
adopted to specify which other properties are held constant. This is
because thermodynamics requires that *two* properties (not one) in
addition to composition information be specified to fix the
intensive state of a substance (or mixture).

Cantera adopts the following convention: only one of the set
(temperature, density, mass fractions) is altered by setting any
single property. In particular:

a) Setting the temperature is done holding density and
   composition  fixed. (The pressure changes.)

b) Setting the pressure is done holding temperature and
   composition fixed. (The density changes.)

c) Setting the composition is done holding temperature
   and density fixed. (The pressure changes).


## Setting multiple properties: 
 ---------------------------------------------

If you want to set multiple properties at once, use 

In [6]:
gas1.TP = 900.0, 1.e5

This statement sets both temperature and pressure at the same
time. Any number of property/value pairs can be specified in the 
"new" Python method 
cf. (http://www.cantera.org/docs/sphinx/html/cython/migrating.html#setting-thermodynamic-state)

The following sets the mole fractions

In [7]:
gas1.TP = 900.0, 1.e5
gas1.X = 'CH4:1,O2:2,N2:7.52'

This all results in 

In [8]:
gas1()


  gri30:

       temperature             900  K
          pressure         7295.13  Pa
           density       0.0269394  kg/m^3
  mean mol. weight         27.6332  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      4.5566e+05        1.259e+07     J
   internal energy      1.8486e+05        5.108e+06     J
           entropy            9317        2.575e+05     J/K
    Gibbs function     -7.9296e+06       -2.191e+08     J
 heat capacity c_p          1304.4        3.604e+04     J/K
 heat capacity c_v          1003.5        2.773e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2       0.190114         0.220149         -30.5775
               CH4       0.095057        0.0551863         -39.6993
                N2       0.714829         0.724665          -27.553
     [  +50 minor]        

Other properties may also be set, including some that
can't be set individually. The following property pairs may be
set: (Enthalpy, Pressure), (IntEnergy, Volume), (Entropy,
Volume), (Entropy, Pressure). In each case, the values of the
extensive properties must be entered *per unit mass*. 

## Setting the enthalpy and pressure:

In [9]:
gas1.HP = 2*gas1.enthalpy_mass, 2*ct.one_atm

The composition above was specified using a string. The format is a
comma-separated list of `<species name>:<relative mole numbers>`
pairs. The mole numbers will be normalized to produce the mole
fractions, and therefore they are 'relative' mole numbers.  Mass
fractions can be set in this way too by changing 'X' to 'Y' in the
above statement.

The composition can also be set using an array, which can be
either a column vector or a row vector but must have the same
size as the number of species. For example, to set all 53 mole
fractions to the same value, do this:

In [10]:
x = np.ones(53) # a column vector of 53 ones
gas1.X = x

To set the mass fractions to equal values:


In [11]:
gas1.Y = x

one doesn't really need to clear up objects created in Python;
please let me know if this is NOT the case @ernestyalumni, otherwise, 
I reference the following:
cf. http://eli.thegreenplace.net/2009/06/12/safely-using-destructors-in-python

######################################################################
#   end of tutorial 1
######################################################################